In [1]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")
import pickle

%reload_ext autoreload
%autoreload 2
from config import keys
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas_profiling
import nltk
import numpy as np
import re
import matplotlib.pyplot as plt 

import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
import unidecode
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
from nltk.stem import PorterStemmer
nltk.download('punkt')
from nltk.tokenize import word_tokenize 
import matplotlib.animation as animation
import operator
import plotly.express as px
from collections import Counter
%matplotlib inline
import string
import contractions
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xiaohansun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/xiaohansun/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/xiaohansun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/xiaohansun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Functions to Obtain Tweets from API 

In [2]:
def group_list(lst, size=100):
    new_list =[]
    idx = 0
    while idx < len(lst):        
        new_list.append(','.join([str(item) for item in lst[idx:idx+size]]))
        idx += size
    return new_list

def tweets_request(tweets_ids):
    lst = []
    for batch in tqdm(tweets_ids):
        url = "https://api.twitter.com/2/tweets?ids={}&&tweet.fields=created_at,entities,geo,id,public_metrics,text&user.fields=description,entities,id,location,name,public_metrics,username".format(batch)
        payload={}
        headers = {'Authorization': 'Bearer ' + keys['bearer_token'],
        'Cookie': 'personalization_id="v1_hzpv7qXpjB6CteyAHDWYQQ=="; guest_id=v1%3A161498381400435837'}
        r = requests.request("GET", url, headers=headers, data=payload)
        data = r.json()
        if 'data' in data.keys():
            lst.append(pd.DataFrame(data['data']))
    return pd.concat(lst)


# Functions for Data Cleaning

### Basic Data Cleaning

In [3]:
def cleaner(tweet):
    tweet = tweet.lower()
    tweet = tweet.strip() #Remove extra space
    tweet = contractions.fix(tweet)
    tweet = re.sub("@[A-Za-z0-9]+:","",tweet) #Remove the username
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = re.sub('(\&amp\;)', '', tweet) #Remove &amp
    tweet = re.sub('(rt)', '', tweet) #Remove "rt"
    tweet = re.sub('(#mkr)', '', tweet) #Remove`@mkr`
    tweet = " ".join(tweet.split())
    return tweet


### Remove Slang Words

In [4]:
#Code for removing slang words
d = {'luv':'love','wud':'would','lyk':'like','wateva':'whatever','ttyl':'talk to you later',
               'kul':'cool','fyn':'fine','omg':'oh my god!','fam':'family','bruh':'brother',
               'cud':'could','fud':'food','lol':'laugh out loud'}

### Remove Punctuations

In [5]:
def removePunctuations(tweet):
    tweet = "".join([char for char in tweet if char not in string.punctuation])
    return tweet

# University of Copenhagen Dataset

### Obtain the Tweets from API 

In [107]:
df_1 = pd.read_csv('data/Original/NAACL_SRW_2016.csv')
df_1

,572342978255048705,racism
0,572341498827522049,racism
1,572340476503724032,racism
2,572334712804384768,racism
3,572332655397629952,racism
4,575949086055997440,racism
...,...,...
16901,576359685843861505,none
16902,576612926838046720,none
16903,576771329975664640,none
16904,560595245814267905,none


In [7]:
df_1.columns = ['id', 'label']
df_1.label.value_counts()

none      11559
sexism     3378
racism     1969
Name: label, dtype: int64

In [8]:
df_racsex = df_1[(df_1['label']=='racism') | (df_1['label']=='sexism')]

In [9]:
df_normal = df_1[(df_1['label']=='none')]

In [10]:
racsex_id = group_list(list(df_racsex.id))

In [11]:
normal_id = group_list(list(df_normal.id))

In [12]:
df_rac_sex = tweets_request(racsex_id)

100%|███████████████████████████████████████████| 54/54 [00:26<00:00,  2.04it/s]


In [13]:
df_rac_sex['label'] = 1
df_1_hate= df_rac_sex[['text', 'label']]

In [14]:
df_normal = tweets_request(normal_id)

100%|█████████████████████████████████████████| 116/116 [00:59<00:00,  1.95it/s]


In [15]:
df_1_hate

,text,label
0,Drasko they didn't cook half a bird you idiot ...,1
1,Hopefully someone cooks Drasko in the next ep ...,1
2,of course you were born in serbia...you're as ...,1
3,These girls are the equivalent of the irritati...,1
0,RT @YesYoureRacist: At least you're only a tin...,1
...,...,...
25,"RT @Superjutah: @MT8_9 £34,000 is £9000 above ...",1
26,RT @DufresneFrank: “@gabystama: @DufresneFrank...,1
27,RT @BoycottBrandy: @ActionFlickDoc this person...,1
28,RT @Lyall: Patriarchy™ will expel me if I divu...,1


In [16]:
df_normal['label'] = 0
df_1_normal = df_normal[['text', 'label']]

In [17]:
df_1_normal.head()

,text,label
0,Someone is going home #mkr ...that obviously c...,0
1,They didn't even wash the chicken 😩 #MKR,0
2,#MKR Is honestly so fucking staged. The most o...,0
3,So bummed the dull chicks on #MKR are using su...,0
4,Can someone smash that bottle of Rose &amp; Li...,0


In [18]:
frames = [df_1_hate, df_1_normal]
  
result = pd.concat(frames)
result = pd.DataFrame(result)
display(result)

,text,label
0,Drasko they didn't cook half a bird you idiot ...,1
1,Hopefully someone cooks Drasko in the next ep ...,1
2,of course you were born in serbia...you're as ...,1
3,These girls are the equivalent of the irritati...,1
0,RT @YesYoureRacist: At least you're only a tin...,1
...,...,...
66,@ProoN ❤️,0
67,RT @Transition: @freebsdgirl just so I’m clear...,0
68,RT @Leonard_Delaney: @freebsdgirl I know! Holy...,0
69,"FLOSS Weekly, open source projects, and paying...",0


In [19]:
result.drop_duplicates(inplace=True)

### Data Cleaning 

In [20]:
# Basic Cleaning
result['clean_tweet'] = result['text']
result['clean_tweet'] = result['clean_tweet'].map(lambda x: cleaner(x))

In [21]:
# Remove All Greek Letters
result['clean_tweet'] = result['clean_tweet'].apply(lambda x : ' '.join([unidecode.unidecode(word) for word in x.split()])) 

In [22]:
# Remove Slang Words
result['clean_tweet'] = result['clean_tweet'].apply(lambda x : ' '.join(d[word] if word in d else word for word in x.split()))

In [23]:
# Find the Hashtags Attached
result['#'] = result['clean_tweet'].apply(lambda x : ' '.join([word for word in x.split() if word.startswith('#')]))
frame = result['#']
frame = pd.DataFrame(frame)
frame = frame.rename({'#':'Hash Words'},axis = 'columns')
frame[frame['Hash Words'] == ''] = 'No hashtags'
frame.head()
data_frame = pd.concat([result,frame],axis = 1)
data_frame.drop('#',axis = 1,inplace = True)
data_frame.head(20)

,text,label,clean_tweet,Hash Words
0,Drasko they didn't cook half a bird you idiot ...,1,drasko they did not cook half a bird you idiot,No hashtags
1,Hopefully someone cooks Drasko in the next ep ...,1,hopefully someone cooks drasko in the next ep of,No hashtags
2,of course you were born in serbia...you're as ...,1,of course you were born in serbia...you are as...,No hashtags
3,These girls are the equivalent of the irritati...,1,these girls are the equivalent of the irritati...,No hashtags
0,RT @YesYoureRacist: At least you're only a tin...,1,at least you are only a tiny bit racist i am n...,No hashtags
1,@MisfitInChains @oldgfatherclock @venereverita...,1,santa just *is* white,No hashtags
2,RT @Dreamdefenders: Eric Holder from #ferguson...,1,"eric holder from #ferguson: ""i understand that...",#ferguson:
3,RT @AntonioFrench: I spent the morning at the ...,1,i spent the morning at the board of elections ...,#ferguson.
4,"RT @harikondabolu: In his #Ferguson testimony,...",1,"in his #ferguson testimony, darren wilson said...",#ferguson
5,"""He can't be a server at our restaurant, that ...",1,"""he cannot be a server at our restaurant, that...",#fuckthanksgiving


In [24]:
# Remove Punctuations
data_frame['clean_tweet'] = data_frame['clean_tweet'].map(lambda x: removePunctuations(x))

In [25]:
# Remove Stopwords
data_frame['clean_tweet'] = data_frame['clean_tweet'].apply(lambda x : ' '.join([word for word in x.split() if not word in set(stopwords.words('english'))]))

In [26]:
# Lemmitization
lemmatizer = WordNetLemmatizer()
data_frame['clean_tweet'] = data_frame['clean_tweet'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

In [27]:
# Stemming
ps = PorterStemmer()

data_frame['clean_tweet'] = data_frame['clean_tweet'].apply(lambda x : ' '.join([ps.stem(word) for word in x.split()]))

In [28]:
data_frame.drop(columns=["text"], inplace=True)

In [29]:
data_frame

,label,clean_tweet,Hash Words
0,1,drasko cook half bird idiot,No hashtags
1,1,hope someon cook drasko next ep,No hashtags
2,1,cours born serbiay fuck serbian film,No hashtags
3,1,girl equival irrit asian girl coupl year ago w...,No hashtags
0,1,least tini bit racist racist dick,No hashtags
...,...,...,...
66,0,,No hashtags
67,0,clear dog name leonard nimoy llap,No hashtags
68,0,know holi crud wohi thank hope enjoy taken tet...,No hashtags
69,0,floss weekli open sourc project pay bill,No hashtags


In [30]:
data_frame['clean_tweet'] = data_frame['clean_tweet'].replace('', np.nan)
data_frame = data_frame.dropna(subset=['clean_tweet'])

In [31]:
data_frame = data_frame.sample(frac=1).reset_index(drop=True)

In [32]:
data_frame_2 = data_frame.drop_duplicates(keep=False)

In [33]:
data_frame_2

,label,clean_tweet,Hash Words
0,0,ok enough instant restaur alreadi,No hashtags
1,0,femal cowork asse woman choos work hooter okay,No hashtags
2,1,call sexist want girl never comment footbal me...,No hashtags
3,0,live long prosper leonard nimoy repo dead 83,No hashtags
5,1,sexist right hate girl,No hashtags
...,...,...,...
9905,0,toni even cross tit,No hashtags
9906,0,let u serv colin chev sausag,No hashtags
9907,0,see guilti look,No hashtags
9908,0,booo kat andr deserv get,No hashtags


In [34]:
data_frame_2.label.value_counts()

0    6698
1    2570
Name: label, dtype: int64

In [35]:
outname = 'Copenhagen_Dataset.csv'

outdir = './data/Cleaning'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

data_frame_2.to_csv(fullname,encoding="utf-8",index=False)

# Hugging Face Dataset

In [36]:
df_2 = pd.read_csv("data/Original/train_tweet.csv", index_col=0)
df_2

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation
...,...,...
31958,0,ate @user isz that youuu?ðððððð...
31959,0,to see nina turner on the airwaves trying to...
31960,0,listening to sad songs on a monday morning otw...


In [37]:
df_2.columns = ['label', 'text']
df_2.label.value_counts()

0    29720
1     2242
Name: label, dtype: int64

### Data Cleaning 

In [38]:
# Basic Cleaning
df_2['clean_tweet'] = df_2['text']
df_2['clean_tweet'] = df_2['clean_tweet'].map(lambda x: cleaner(x))

In [39]:
# Remove All Greek Letters
df_2['clean_tweet'] = df_2['clean_tweet'].apply(lambda x : ' '.join([unidecode.unidecode(word) for word in x.split()])) 

In [40]:
# Remove Slang Words
df_2['clean_tweet'] = df_2['clean_tweet'].apply(lambda x : ' '.join(d[word] if word in d else word for word in x.split()))

In [41]:
# Find the Hashtags Attached
df_2['#'] = df_2['clean_tweet'].apply(lambda x : ' '.join([word for word in x.split() if word.startswith('#')]))
frame = df_2['#']
frame = pd.DataFrame(frame)
frame = frame.rename({'#':'Hash Words'},axis = 'columns')
frame[frame['Hash Words'] == ''] = 'No hashtags'
frame.head()


,Hash Words
id,
1,#run
2,#lyft #disapointed #getthanked
3,No hashtags
4,#model
5,#motivation


In [42]:
huggingFace = pd.concat([df_2,frame],axis = 1)
huggingFace.drop('#',axis = 1,inplace = True)
huggingFace.head(20)

,label,text,clean_tweet,Hash Words
id,,,,
1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...,#run
2,0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i cannot use because t...,#lyft #disapointed #getthanked
3,0,bihday your majesty,bihday your majesty,No hashtags
4,0,#model i love u take with u all the time in ...,#model i love you take with you all the time i...,#model
5,0,factsguide: society now #motivation,factsguide: society now #motivation,#motivation
6,0,[2/2] huge fan fare and big talking before the...,[2/2] huge fan fare and big talking before the...,#allshowandnogo
7,0,@user camping tomorrow @user @user @user @use...,camping tomorrow dannya|,No hashtags
8,0,the next school year is the year for exams.ð...,the next school year is the year for exams.d- ...,#school #exams #hate #imagine #actorslife #rev...
9,0,we won!!! love the land!!! #allin #cavs #champ...,we won!!! love the land!!! #allin #cavs #champ...,#allin #cavs #champions #cleveland #clevelandc...


In [43]:
display(huggingFace.dtypes)

label           int64
text           object
clean_tweet    object
Hash Words     object
dtype: object

In [44]:
# Remove Punctuations
huggingFace['clean_tweet'] = huggingFace['clean_tweet'].map(lambda x: removePunctuations(x))

In [45]:
# Remove Stopwords
huggingFace['clean_tweet'] = huggingFace['clean_tweet'].apply(lambda x : ' '.join([word for word in x.split() if not word in set(stopwords.words('english'))]))

In [46]:
# Lemmitization
lemmatizer = WordNetLemmatizer()
huggingFace['clean_tweet'] = huggingFace['clean_tweet'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

In [47]:
# Stemming
ps = PorterStemmer()

huggingFace['clean_tweet'] = huggingFace['clean_tweet'].apply(lambda x : ' '.join([ps.stem(word) for word in x.split()]))

In [48]:
huggingFace.drop(columns=["text"], inplace=True)

In [49]:
huggingFace['clean_tweet'] = huggingFace['clean_tweet'].replace('', np.nan)
huggingFace = huggingFace.dropna(subset=['clean_tweet'])

In [50]:
huggingFace.label.value_counts()

0    29698
1     2239
Name: label, dtype: int64

In [51]:
huggingFace = huggingFace.sample(frac=1).reset_index(drop=True)

In [52]:
huggingFace

,label,clean_tweet,Hash Words
0,0,spread positiv give someon high five today pos...,#positiveenergy #highfive #tuesdaymotivation
1,0,difficult ask mani languag measur happi unifor...,#languages- #happiness
2,0,got ticket 12th juli,No hashtags
3,0,pray orlando orlando orlandostrong bestrong pr...,#orlando #orlandostrong #bestrong #pray #belie...
4,0,bought thing kyli cosmet today ddd lovethemal,#lovethemall
...,...,...,...
31932,0,goodnight sweet dream grate ddd,#grateful
31933,0,belum ku liat si dd mayb soon insha allah go w...,No hashtags
31934,0,cheer sexysunday onset sexi rileysteel dress s...,#sexysunday #onset #sexy #rileysteele #dress #...
31935,0,oooooo go soo jasonbourn back,#jasonbourne


In [53]:
huggingFace_2 = huggingFace.drop_duplicates(keep=False)

In [54]:
huggingFace_2

,label,clean_tweet,Hash Words
0,0,spread positiv give someon high five today pos...,#positiveenergy #highfive #tuesdaymotivation
1,0,difficult ask mani languag measur happi unifor...,#languages- #happiness
3,0,pray orlando orlando orlandostrong bestrong pr...,#orlando #orlandostrong #bestrong #pray #belie...
4,0,bought thing kyli cosmet today ddd lovethemal,#lovethemall
5,0,gorgeou modeltoi look wild sexi follow victori...,#model
...,...,...,...
31932,0,goodnight sweet dream grate ddd,#grateful
31933,0,belum ku liat si dd mayb soon insha allah go w...,No hashtags
31934,0,cheer sexysunday onset sexi rileysteel dress s...,#sexysunday #onset #sexy #rileysteele #dress #...
31935,0,oooooo go soo jasonbourn back,#jasonbourne


In [55]:
huggingFace_2.label.value_counts()

0    25255
1     1676
Name: label, dtype: int64

In [56]:
outname = 'HuggingFace_Dataset.csv'

outdir = './data/Cleaning'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

huggingFace_2.to_csv(fullname,encoding="utf-8",index=False)

# Aristotle Dataset

In [57]:
df_3 = pd.read_csv("data/Original/hatespeechtwitter.csv")
df_3.head()

,tweet_id,maj_label
0,849667487180259329,abusive
1,850490912954351616,abusive
2,848791766853668864,abusive
3,848306464892604416,abusive
4,850010509969465344,normal


In [58]:
df_3.columns = ['id', 'label']
df_3.label.value_counts()

normal     52835
spam       13404
abusive    10122
hateful     3635
Name: label, dtype: int64

In [59]:
# setup url and headers for calling Twitter API as per documentation
url = "https://api.twitter.com/2/tweets?ids=847661947159891972,847799130277675008,848933211375779840&tweet.fields=created_at,entities,geo,id,public_metrics,text&user.fields=description,entities,id,location,name,public_metrics,username"
payload={}
headers = {'Authorization': 'Bearer ' + keys['bearer_token'], 'Cookie': 'personalization_id="v1_hzpv7qXpjB6CteyAHDWYQQ=="; guest_id=v1%3A161498381400435837'}

# make GET request to Twitter API to get response object 
r = requests.request("GET", url, headers=headers, data=payload)

# convert payload to dictionary
data = r.json()

In [60]:
df_hateful = df_3[df_3['label']=='hateful']
hate_ids = group_list(list(df_hateful.id))

In [61]:
df_normal = df_3[df_3['label']=='normal']
normal_ids = group_list(list(df_normal.id))

In [62]:
df_hate = tweets_request(hate_ids)

100%|███████████████████████████████████████████| 37/37 [00:19<00:00,  1.87it/s]


In [63]:
df_normal = tweets_request(normal_ids)

100%|█████████████████████████████████████████| 529/529 [02:34<00:00,  3.42it/s]


In [64]:
df_hate['label'] = 1
df_3_hate = df_hate[['text', 'label']]
df_3_hate.count()

text     1621
label    1621
dtype: int64

In [65]:
df_normal['label'] = 0
df_3_normal = df_normal[['text', 'label']]
df_3_normal.count()

text     5632
label    5632
dtype: int64

In [66]:
frames = [df_3_hate, df_3_normal]
  
result = pd.concat(frames)
display(result)

,text,label
0,I hate er chase because if the Bitch that work...,1
1,RT @nyctophil3: Pineapples do not belong on pi...,1
2,Niggas keep talking about women wearing weave ...,1
3,@vappywave idiot that's not gonna work. you go...,1
4,RT @ayevonnn: bruh i fucking hate people like ...,1
...,...,...
56,"@realDonaldTrump Just keep golfing, instead of...",0
57,"D'banj conquered wen dia wasn't social media, ...",0
58,@JillStein4Prez Perj @LSteinRoeder @regstein m...,0
59,"@hamboman Hi Hamish, amendments to orders can ...",0


In [67]:
result.drop_duplicates(inplace=True)

### Data Cleaning 

In [68]:
# Basic Cleaning
result['clean_tweet'] = result['text']
result['clean_tweet'] = result['clean_tweet'].map(lambda x: cleaner(x))

In [69]:
# Remove All Greek Letters
result['clean_tweet'] = result['clean_tweet'].apply(lambda x : ' '.join([unidecode.unidecode(word) for word in x.split()])) 

In [70]:
# Remove Slang Words
result['clean_tweet'] = result['clean_tweet'].apply(lambda x : ' '.join(d[word] if word in d else word for word in x.split()))

In [71]:
result

,text,label,clean_tweet
0,I hate er chase because if the Bitch that work...,1,i hate er chase because if the bitch that work...
1,RT @nyctophil3: Pineapples do not belong on pi...,1,pineapples do not belong on pizza. you all nig...
2,Niggas keep talking about women wearing weave ...,1,niggas keep talking about women wearing weave ...
3,@vappywave idiot that's not gonna work. you go...,1,idiot that is not going to work. you got to pu...
4,RT @ayevonnn: bruh i fucking hate people like ...,1,brother i fucking hate people like this
...,...,...,...
56,"@realDonaldTrump Just keep golfing, instead of...",0,"just keep golfing, instead of working, ya know..."
57,"D'banj conquered wen dia wasn't social media, ...",0,"d'banj conquered wen dia was not social media,..."
58,@JillStein4Prez Perj @LSteinRoeder @regstein m...,0,perj misrep 0%execpower #regireg as madamsecre...
59,"@hamboman Hi Hamish, amendments to orders can ...",0,"hi hamish, amendments to orders can only be ma..."


In [72]:
# Find the Hashtags Attached
result['#'] = result['clean_tweet'].apply(lambda x : ' '.join([word for word in x.split() if word.startswith('#')]))
frame = result['#']
frame = pd.DataFrame(frame)
frame = frame.rename({'#':'Hash Words'},axis = 'columns')
frame[frame['Hash Words'] == ''] = 'No hashtags'
frame.head()
aristotle = pd.concat([result,frame],axis = 1)
aristotle.drop('#',axis = 1,inplace = True)
aristotle.head(20)

,text,label,clean_tweet,Hash Words
0,I hate er chase because if the Bitch that work...,1,i hate er chase because if the bitch that work...,No hashtags
1,RT @nyctophil3: Pineapples do not belong on pi...,1,pineapples do not belong on pizza. you all nig...,No hashtags
2,Niggas keep talking about women wearing weave ...,1,niggas keep talking about women wearing weave ...,No hashtags
3,@vappywave idiot that's not gonna work. you go...,1,idiot that is not going to work. you got to pu...,No hashtags
4,RT @ayevonnn: bruh i fucking hate people like ...,1,brother i fucking hate people like this,No hashtags
5,RT @mattmfm: I'm really fucking sick of watchi...,1,i am really fucking sick of watching the repub...,No hashtags
6,@JayFoee_ just another dumbass bronco fan swea...,1,just another dumbass bronco fan swear i hate t...,No hashtags
7,RT @THESLUMPGOD: I Sampled Jaws \n\nPull Up Wi...,1,i sampled jaws pull up with the pistol make a ...,No hashtags
8,RT @arianam0lina: if you litter you're a bitch...,1,if you litter you are a bitch i hate you,No hashtags
9,He would not have won if the DNC knew what the...,1,he would not have won if the dnc knew what the...,No hashtags


In [73]:
# Remove Punctuations
aristotle['clean_tweet'] = aristotle['clean_tweet'].map(lambda x: removePunctuations(x))

In [74]:
# Remove Stopwords
aristotle['clean_tweet'] = aristotle['clean_tweet'].apply(lambda x : ' '.join([word for word in x.split() if not word in set(stopwords.words('english'))]))

In [75]:
# Lemmitization
lemmatizer = WordNetLemmatizer()
aristotle['clean_tweet'] = aristotle['clean_tweet'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

In [76]:
# Stemming
ps = PorterStemmer()

aristotle['clean_tweet'] = aristotle['clean_tweet'].apply(lambda x : ' '.join([ps.stem(word) for word in x.split()]))

In [77]:
aristotle.drop(columns=["text"], inplace=True)

In [78]:
aristotle['clean_tweet'] = aristotle['clean_tweet'].replace('', np.nan)
aristotle = aristotle.dropna(subset=['clean_tweet'])

In [79]:
aristotle 

,label,clean_tweet,Hash Words
0,1,hate er chase bitch work liter evil,No hashtags
1,1,pineappl belong pizza nigga nasti,No hashtags
2,1,nigga keep talk woman wear weav sick bitch fro as,No hashtags
3,1,idiot go work got put flame sticker make go fa...,No hashtags
4,1,brother fuck hate peopl like,No hashtags
...,...,...,...
56,0,keep golf instead work ya know leader kind thi...,No hashtags
57,0,dbanj conquer wen dia social medium diaz socia...,No hashtags
58,0,perj misrep 0execpow regireg madamsecretoffice...,#regireg
59,0,hi hamish amend order made within 15 minut ord...,No hashtags


In [80]:
aristotle = aristotle.sample(frac=1).reset_index(drop=True)

In [81]:
aristotle_2 = aristotle.drop_duplicates(keep=False)

In [82]:
aristotle_2

,label,clean_tweet,Hash Words
0,0,take shot predict bengal first two draft day i...,#bengals
1,0,next week localnewspap shop wednesday afternoo...,#localnewspaper
2,0,final get player dnd soon align go think chaot...,No hashtags
3,0,commit oral write idea goal like honor commitm...,No hashtags
4,1,syrian alli iran blast you missil strike dange...,No hashtags
...,...,...,...
7121,1,scold two littl cousin scare friend poor littl...,No hashtags
7122,0,spent afternoon w 4 ucalgari theatr student jo...,No hashtags
7123,0,ask pleat 5 nill today 6 scot striker english ...,No hashtags
7124,0,petey green tutor program info session campu e...,No hashtags


In [83]:
aristotle_2.label.value_counts()

0    5352
1    1538
Name: label, dtype: int64

In [84]:
outname = 'Aristotle_Dataset.csv'

outdir = './data/Cleaning'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

aristotle_2.to_csv(fullname,encoding="utf-8",index=False)

# HASOC Dataset

In [85]:
df_4 = pd.read_csv("data/Original/english_dataset.tsv", delimiter = "\t")
df_4.head(20)

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE
5,hasoc_en_6,@kellymiller513 @TheRealOJ32 I hope you rememb...,NOT,NONE,NONE
6,hasoc_en_7,@ICC Latest design of #WC2019 trophy. #CWC2019...,NOT,NONE,NONE
7,hasoc_en_8,#ADOS #trendingnow #blacklivesmatter #justice ...,HOF,PRFN,TIN
8,hasoc_en_9,Thanks for your support! Wow 600k. Graffiti ha...,NOT,NONE,NONE
9,hasoc_en_10,By wearing the #BalidaanBadge over his gloves ...,NOT,NONE,NONE


In [86]:
df_4_hate = df_4[df_4.task_2 == 'HATE']
df_4_hate['label'] = 1
df_4_hate= df_4_hate[['text', 'label']]
df_4_hate.head()

,text,label
1,@politico No. We should remember very clearly ...,1
11,I don’t know how much more I can take! 45 is a...,1
23,#ShameOnICC 1. ICC on Dhoni's gloves ...,1
27,Disgusting! And they don’t change their cultur...,1
34,"@TheRealOJ32 Of all the retired NFL players, w...",1


In [87]:
df_4_normal = df_4.loc[(df_4['task_1']=="NOT") & (df_4['task_2']=="NONE") & (df_4['task_1']=="NOT")]
df_4_normal.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE
5,hasoc_en_6,@kellymiller513 @TheRealOJ32 I hope you rememb...,NOT,NONE,NONE


In [88]:
df_4_normal['label'] = 0
df_4_normal= df_4_normal[['text', 'label']]
df_4_normal.head()

,text,label
0,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,0
2,@cricketworldcup Guess who would be the winner...,0
3,Corbyn is too politically intellectual for #Bo...,0
4,All the best to #TeamIndia for another swimmin...,0
5,@kellymiller513 @TheRealOJ32 I hope you rememb...,0


In [89]:
frames = [df_4_hate, df_4_normal]
  
result = pd.concat(frames)
display(result)

,text,label
1,@politico No. We should remember very clearly ...,1
11,I don’t know how much more I can take! 45 is a...,1
23,#ShameOnICC 1. ICC on Dhoni's gloves ...,1
27,Disgusting! And they don’t change their cultur...,1
34,"@TheRealOJ32 Of all the retired NFL players, w...",1
...,...,...
5824,@DOlusegun @elrufai Hope they don't becomes #D...,0
5825,@gtconway3d @KellyannePolls @realDonaldTrump H...,0
5827,@washingtonpost This is something everyone sho...,0
5828,Sorry about that. Has anybody got a #dickhead ...,0


### Data Cleaning 

In [90]:
# Basic Cleaning
result['clean_tweet'] = result['text']
result['clean_tweet'] = result['clean_tweet'].map(lambda x: cleaner(x))

In [91]:
# Remove All Greek Letters
result['clean_tweet'] = result['clean_tweet'].apply(lambda x : ' '.join([unidecode.unidecode(word) for word in x.split()])) 

In [92]:
# Remove Slang Words
result['clean_tweet'] = result['clean_tweet'].apply(lambda x : ' '.join(d[word] if word in d else word for word in x.split()))

In [93]:
# Find the Hashtags Attached
result['#'] = result['clean_tweet'].apply(lambda x : ' '.join([word for word in x.split() if word.startswith('#')]))
frame = result['#']
frame = pd.DataFrame(frame)
frame = frame.rename({'#':'Hash Words'},axis = 'columns')
frame[frame['Hash Words'] == ''] = 'No hashtags'
frame.head()
hasoc = pd.concat([result,frame],axis = 1)
hasoc.drop('#',axis = 1,inplace = True)
hasoc.head(20)

,text,label,clean_tweet,Hash Words
1,@politico No. We should remember very clearly ...,1,no. we should remember very clearly that #indi...,#individual1 #trumpisatraitor #mccainsahero #j...
11,I don’t know how much more I can take! 45 is a...,1,i do not know how much more i can take! 45 is ...,#trump30hours #trumpisatraitor
23,#ShameOnICC 1. ICC on Dhoni's gloves ...,1,#shameonicc 1. icc on dhoni's gloves vs 2.icc ...,#shameonicc
27,Disgusting! And they don’t change their cultur...,1,disgusting! and they do not change their cultu...,#rapist #childtrafficking #catholicchurch #cat...
34,"@TheRealOJ32 Of all the retired NFL players, w...",1,"of all the retired nfl players, why is it that...",#murderer
95,@kojikabu @Maggie_Klaus @JoeBiden What it is t...,1,what it is that we will do anything to get tru...,#fucktrump #bluewave2020
311,#ShameOnICC ICC is earning billions from this...,1,#shameonicc icc is earning billions from this ...,#shameonicc
371,Its a shame when #antimuslim #Islamophobe @rea...,1,its a shame when #antimuslim #islamophobe is a...,#antimuslim #islamophobe #istandwithsadiqkhan ...
372,@funder Am I the only one who feels like it’s ...,1,am i the only one who feels like it is no long...,"#trumpisatraitor,"
379,"'Well, even his “friendly” Fox &amp; Friends w...",1,"'well, even his ""friendly"" fox friends were on...",#trumpisatraitor #trumpresignnow #trump #trump...


In [94]:
# Remove Punctuations
hasoc['clean_tweet'] = hasoc['clean_tweet'].map(lambda x: removePunctuations(x))

In [95]:
# Remove Stopwords
hasoc['clean_tweet'] = hasoc['clean_tweet'].apply(lambda x : ' '.join([word for word in x.split() if not word in set(stopwords.words('english'))]))

In [96]:
# Lemmitization
lemmatizer = WordNetLemmatizer()
hasoc['clean_tweet'] = hasoc['clean_tweet'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

In [97]:
# Stemming
ps = PorterStemmer()

hasoc['clean_tweet'] = hasoc['clean_tweet'].apply(lambda x : ' '.join([ps.stem(word) for word in x.split()]))

In [98]:
hasoc

,text,label,clean_tweet,Hash Words
1,@politico No. We should remember very clearly ...,1,rememb clearli individual1 admit treason trump...,#individual1 #trumpisatraitor #mccainsahero #j...
11,I don’t know how much more I can take! 45 is a...,1,know much take 45 compuls liar trump30hour tru...,#trump30hours #trumpisatraitor
23,#ShameOnICC 1. ICC on Dhoni's gloves ...,1,shameonicc 1 icc dhoni glove v 2icc plan world...,#shameonicc
27,Disgusting! And they don’t change their cultur...,1,disgust chang cultur come vet system prevent w...,#rapist #childtrafficking #catholicchurch #cat...
34,"@TheRealOJ32 Of all the retired NFL players, w...",1,retir nfl player suffer cte bottom pool mistoo...,#murderer
...,...,...,...,...
5824,@DOlusegun @elrufai Hope they don't becomes #D...,0,hope becom dickhead,#dickhead
5825,@gtconway3d @KellyannePolls @realDonaldTrump H...,0,hey georg wife lawyer right risk disbar violat...,#trumpisatraitor
5827,@washingtonpost This is something everyone sho...,0,someth everyon worri trump joke intent leav do...,#donthecon #fraud #cheater #liar #traitor #lia...
5828,Sorry about that. Has anybody got a #dickhead ...,0,sorri anybodi got dickhead emoji tri search em...,#dickhead


In [99]:
hasoc.drop(columns=["text"], inplace=True)

In [100]:
hasoc['clean_tweet'] = hasoc['clean_tweet'].replace('', np.nan)
hasoc = hasoc.dropna(subset=['clean_tweet'])

In [101]:
hasoc = hasoc.sample(frac=1).reset_index(drop=True)

In [102]:
hasoc

,label,clean_tweet,Hash Words
0,0,lulz new meme day opkkk optrump opdomesticterr...,#opkkk #optrump #opdomesticterrorism #fucktrum...
1,1,gandinaaliabus arnob r great job showcas genui...,#gandinaaliabuse
2,0,enough harass still like unwelcom rel leav eve...,#johnmccainday #trumpisatraitor #collusion
3,0,3rd class organ world biggest cup sad see help...,#icc #shameonicc
4,0,hey love india cc dhonikeepstheglov balidaanba...,#dhonikeepstheglove #balidaanbadge #dhonikesaa...
...,...,...,...
4729,0,leader free world find need even care mayor ci...,#pathetic #sociopathic #douchebag
4730,0,befit repli section medium look controversi dh...,#dhonikeepstheglove #dhonigloves
4731,0,heard pa clear trumpisatraitor trumplieseveryt...,#trumpisatraitor #trumplieseverytimehespeaks #...
4732,0,celebr putinspuppet desperatedonald trumpisatr...,#putinspuppet #desperatedonald #trumpisatraito...


In [103]:
hasoc_2 = hasoc.drop_duplicates(keep=False)

In [104]:
hasoc_2

,label,clean_tweet,Hash Words
0,0,lulz new meme day opkkk optrump opdomesticterr...,#opkkk #optrump #opdomesticterrorism #fucktrum...
1,1,gandinaaliabus arnob r great job showcas genui...,#gandinaaliabuse
2,0,enough harass still like unwelcom rel leav eve...,#johnmccainday #trumpisatraitor #collusion
3,0,3rd class organ world biggest cup sad see help...,#icc #shameonicc
4,0,hey love india cc dhonikeepstheglov balidaanba...,#dhonikeepstheglove #balidaanbadge #dhonikesaa...
...,...,...,...
4729,0,leader free world find need even care mayor ci...,#pathetic #sociopathic #douchebag
4730,0,befit repli section medium look controversi dh...,#dhonikeepstheglove #dhonigloves
4731,0,heard pa clear trumpisatraitor trumplieseveryt...,#trumpisatraitor #trumplieseverytimehespeaks #...
4732,0,celebr putinspuppet desperatedonald trumpisatr...,#putinspuppet #desperatedonald #trumpisatraito...


In [105]:
hasoc_2.label.value_counts()

0    3419
1    1122
Name: label, dtype: int64

In [106]:
outname = 'HASOC_Dataset.csv'

outdir = './data/Cleaning'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    
hasoc_2.to_csv(fullname,encoding="utf-8",index=False)